In [1]:
import cv2
from shapely.geometry import Polygon

from PIL import Image
import numpy as np
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision import models
from torch import nn

import pandas as pd
from PIL import ImageEnhance

import torch
from torchvision import models
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax
from torch.optim import Adam,lr_scheduler
from torch.nn import functional as F
from torchvision import transforms
from torch.utils.data import Dataset

In [ ]:
Deeplabv3plus_path = r'E:\codes\python\area51m'
Data_path = r'D:\Documents\GitHub\plant-segmentation'

ModelPath = r'E:\data\MODELS' 

OneStageModelName = 'model_deeplabv3+_resnet_onestage'
TwoStageModelName = 'model_deeplabv3+_resnet_twostage'

SfMPath = r'E:\data\testagain2'
OriginalImageFolder = r'E:\data\jpgimages'

SfMSparseModel = 'sparsemodel.nvm'
SfMTextureFolder = 'textures'
OutputFolder = 'superimposedresults'

OpenMVSPath = r'C:\Users\assembled\OpenMVS\src\bin\vc15\x64\Release'
VisualSFMPath = r'F:\cx\VisualSFM_windows_cuda_64bit'

In [ ]:
from utils import *
import sys
import os

device = torch.device("cuda:0")

os.environ['TORCH_HOME'] = ModelPath

In [ ]:
dlab2,transform_deeplab = load_model(Number_of_Class = 8,
                                     Image_size = (900,900),
                                     Backbone = 'resnet',
                                     Path = ModelPath,
                                     Name = OneStageModelName)

dlab2_detail,transform_deeplab_detail = load_model(Number_of_Class = 2,
                                                   Image_size = (200,200),
                                                   Backbone = 'resnet',
                                                   Path = ModelPath,
                                                   Name = TwoStageModelName)

In [56]:
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)
yellow = (255,225,0)
orange = (255,125,0)
purple = (155,0,255)
greenblue = (0,255,225)
pink = (255,140,248)
white = (255,255,255)
black = (0,0,0)
colors = [red,green,blue,yellow,orange,purple,greenblue,pink]

In [ ]:
if os.path.exists(SfMPath) == False:
    os.mkdir(SfMPath)
    
if os.path.exists(r'{0}\{1}'.format(SfMPath,SfMTextureFolder)) == False:
    os.mkdir(r'{0}\{1}'.format(SfMPath,SfMTextureFolder))

for i in os.listdir(OriginalImageFolder):
    shutil.copy(r'{0}\{1}'.format(OriginalImageFolder,i),r'{0}\{1}\{2}'.format(SfMPath,SfMTextureFolder,i))

os.chdir(VisualSFMPath)

os.popen(r'VisualSFM sfm {0}\{1} {0}\{2}'.format(SfMPath,SfMTextureFolder,SfMSparseModel.split('.')[0])).read()

print('finished Visual SFM')

making_masks(colors = 'binary',
             mask_alpha = 0.5,
             Path = SfMPath,
             InputFolder = OriginalImageFolder,
             OutputFolder = OutputFolder,
             transform_deeplab = transform_deeplab,
             transform_deeplab_detail = transform_deeplab_detail,
             dlab2 = dlab2,
             dlab2_detail = dlab2_detail,
             class_label = 3,
             two_stage = True,
             scale = 0.05)

print('finished making masks')

os.chdir(OpenMVSPath)

os.popen(r'InterfaceVisualSFM -w {0} {1}'.format(SfMPath,SfMSparseModel)).read()

print('finished reading {0}'.format(SfMSparseModel))

os.popen(r'ReconstructMesh -w {0} {1}.mvs'.format(SfMPath,SfMSparseModel.split('.')[0])).read()

print('finished reconstructing mesh')

for i in os.listdir(r'{0}\{1}'.format(SfMPath,OutputFolder)):
    shutil.copy(r'{0}\{1}\{2}'.format(SfMPath,OutputFolder,i),r'{0}\{1}\{2}'.format(SfMPath,SfMTextureFolder,i))
    
os.popen(r'TextureMesh -w {0} {1}_mesh.mvs'.format(SfMPath,SfMSparseModel.split('.')[0])).read()

print('finished texturing')